In [15]:
import pickle as pkl
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm

from scipy import spatial
from scipy.stats import weightedtau

from pandas.core.common import SettingWithCopyWarning
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
# season = '2020-21'
all_players = pd.read_csv("data/all_players.csv")
all_teams = pd.read_csv("data/all_teams.csv")
merged = pd.read_csv("data/merged.csv")
# merged = merged[merged["SEASON_ID"] == season]
team_stats = merged.groupby("TEAM")[
    [
        "FGM",
        "FGA",
        "FG3M",
        "FG3A",
        "FTM",
        "FTA",
        "OREB",
        "DREB",
        "REB",
        "AST",
        "STL",
        "BLK",
        "TOV",
        "PF",
        "PTS",
    ]
].sum()
with open("data/matches.pkl", "rb") as file:
    matches = pkl.load(file)

In [5]:
# game played
mtch = pd.DataFrame()
for key in matches.keys():
    temp = matches[key]
    mtch = mtch.append([temp])

In [6]:
agg = mtch.groupby(['GAME_ID','TEAM_ID','GAME_DATE']).sum().reset_index()
agg = agg.merge(all_teams, left_on='TEAM_ID', right_on='id', how='left')
agg.drop(['id','nickname','city','state','year_founded'], axis=1, inplace=True)

In [7]:
test = agg.merge(agg, on='GAME_ID', how='inner')
test = test[~(test.TEAM_ID_x == test.TEAM_ID_y)]
test = test.iloc[::2,:]
test.drop(['GAME_DATE_y'], axis=1, inplace=True)

In [8]:
def col_fixer(col):
    if 'x' in col:
        return f"home_{col[:-2]}"
    if 'y' in col:
        return f"away_{col[:-2]}"
    else:
        return col

In [9]:
test.columns = list(map(col_fixer, test.columns))

In [10]:
test.head()

,GAME_ID,home_TEAM_ID,home_GAME_DATE,home_MIN,home_PTS,home_FGM,home_FGA,home_FG_PCT,home_FG3M,home_FG3A,home_FG3_PCT,home_FTM,home_FTA,home_FT_PCT,home_OREB,home_DREB,home_REB,home_AST,home_STL,home_BLK,home_TOV,home_PF,home_PLUS_MINUS,home_full_name,home_abbreviation,away_TEAM_ID,away_MIN,away_PTS,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FTM,away_FTA,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_STL,away_BLK,away_TOV,away_PF,away_PLUS_MINUS,away_full_name,away_abbreviation
1,0010300001,1610612742,2003-10-05,55,85,34,76,0.447,2,8.0,0.250,15,30,0.500,12.0,26.0,38.0,20,9.0,4,18,34,0.0,Dallas Mavericks,DAL,1610612762,95,90,32,70,0.457,1,7.0,0.143,25,34,0.735,9.0,32.0,41.0,23,8.0,4,18,26,0.0,Utah Jazz,UTA
5,0010300002,1610612749,2003-10-06,65,94,32,75,0.427,2,13.0,0.154,28,40,0.700,11.0,32.0,43.0,20,9.0,4,24,26,0.0,Milwaukee Bucks,MIL,1610612763,75,105,40,81,0.494,4,15.0,0.267,21,34,0.618,14.0,34.0,48.0,25,18.0,7,25,35,0.0,Memphis Grizzlies,MEM
9,0010300003,1610612739,2003-10-07,70,100,38,77,0.494,4,6.0,0.667,20,29,0.690,12.0,40.0,52.0,25,10.0,7,25,33,0.0,Cleveland Cavaliers,CLE,1610612765,70,96,34,87,0.391,4,9.0,0.444,24,40,0.600,12.0,25.0,37.0,19,15.0,3,13,27,0.0,Detroit Pistons,DET
13,0010300004,1610612742,2003-10-07,65,99,40,82,0.488,1,6.0,0.167,18,27,0.667,15.0,37.0,52.0,20,9.0,5,17,30,0.0,Dallas Mavericks,DAL,1610612753,65,76,25,61,0.410,6,18.0,0.333,20,24,0.833,6.0,23.0,29.0,14,9.0,5,19,24,0.0,Orlando Magic,ORL
17,0010300005,1610612745,2003-10-07,60,78,30,64,0.469,4,12.0,0.333,14,21,0.667,12.0,22.0,34.0,10,9.0,2,29,25,0.0,Houston Rockets,HOU,1610612757,65,102,38,70,0.543,3,7.0,0.429,23,35,0.657,10.0,22.0,32.0,21,17.0,3,15,22,0.0,Portland Trail Blazers,POR


In [19]:
test.home_full_name.nunique()

29

In [20]:
test.away_full_name.nunique()

29

In [5]:
mtch.shape

(50279, 28)

In [6]:
mtch['opponent'] = mtch.MATCHUP.apply(lambda x: x[-3:])

In [7]:
sub_df = mtch[mtch.groupby('opponent').opponent.transform('count')>21].copy() 
sub_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,opponent
0,2021-22,1610612737,ATL,Atlanta Hawks,0022100670,2022-01-19,ATL vs. MIN,W,241,134,45,86,0.523,15,38.0,0.395,29,33,0.879,8.0,31.0,39.0,34,7.0,9,11,23,12.0,MIN
1,2021-22,1610612737,ATL,Atlanta Hawks,0022100660,2022-01-17,ATL vs. MIL,W,242,121,38,86,0.442,15,36.0,0.417,30,32,0.938,10.0,36.0,46.0,25,7.0,8,13,18,7.0,MIL
2,2021-22,1610612737,ATL,Atlanta Hawks,0022100643,2022-01-15,ATL vs. NYK,L,240,108,40,80,0.500,16,44.0,0.364,12,17,0.706,4.0,27.0,31.0,23,5.0,5,9,20,-9.0,NYK
3,2021-22,1610612737,ATL,Atlanta Hawks,0022100636,2022-01-14,ATL @ MIA,L,240,118,38,79,0.481,16,38.0,0.421,26,30,0.867,9.0,25.0,34.0,26,6.0,6,15,27,-6.0,MIA
4,2021-22,1610612737,ATL,Atlanta Hawks,0022100621,2022-01-12,ATL vs. MIA,L,239,91,31,82,0.378,13,45.0,0.289,16,23,0.696,6.0,31.0,37.0,20,8.0,3,14,15,-24.0,MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,2004-05,1610612766,CHA,Charlotte Bobcats,0020400031,2004-11-06,CHA vs. ORL,W,240,111,40,88,0.455,3,4.0,0.750,28,43,0.651,13.0,36.0,49.0,23,7.0,6,14,29,11.0,ORL
1581,2004-05,1610612766,CHA,Charlotte Bobcats,0020400017,2004-11-04,CHA vs. WAS,L,240,96,37,78,0.474,1,8.0,0.125,21,32,0.656,14.0,28.0,42.0,22,6.0,5,19,27,-7.0,WAS
1582,2004-05,1610612766,CHA,Charlotte Bobcats,0010400091,2004-10-26,CHA vs. MIA,W,60,82,33,74,0.446,4,7.0,0.571,12,16,0.750,9.0,32.0,41.0,19,5.0,4,11,25,NaN,MIA
1583,2004-05,1610612766,CHA,Charlotte Bobcats,0010400084,2004-10-24,CHA @ WAS,L,60,85,32,72,0.444,3,12.0,0.250,18,24,0.750,12.0,26.0,38.0,19,12.0,1,24,37,NaN,WAS


In [8]:
sub_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,opponent
0,2021-22,1610612737,ATL,Atlanta Hawks,0022100670,2022-01-19,ATL vs. MIN,W,241,134,45,86,0.523,15,38.0,0.395,29,33,0.879,8.0,31.0,39.0,34,7.0,9,11,23,12.0,MIN
1,2021-22,1610612737,ATL,Atlanta Hawks,0022100660,2022-01-17,ATL vs. MIL,W,242,121,38,86,0.442,15,36.0,0.417,30,32,0.938,10.0,36.0,46.0,25,7.0,8,13,18,7.0,MIL
2,2021-22,1610612737,ATL,Atlanta Hawks,0022100643,2022-01-15,ATL vs. NYK,L,240,108,40,80,0.500,16,44.0,0.364,12,17,0.706,4.0,27.0,31.0,23,5.0,5,9,20,-9.0,NYK
3,2021-22,1610612737,ATL,Atlanta Hawks,0022100636,2022-01-14,ATL @ MIA,L,240,118,38,79,0.481,16,38.0,0.421,26,30,0.867,9.0,25.0,34.0,26,6.0,6,15,27,-6.0,MIA
4,2021-22,1610612737,ATL,Atlanta Hawks,0022100621,2022-01-12,ATL vs. MIA,L,239,91,31,82,0.378,13,45.0,0.289,16,23,0.696,6.0,31.0,37.0,20,8.0,3,14,15,-24.0,MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1580,2004-05,1610612766,CHA,Charlotte Bobcats,0020400031,2004-11-06,CHA vs. ORL,W,240,111,40,88,0.455,3,4.0,0.750,28,43,0.651,13.0,36.0,49.0,23,7.0,6,14,29,11.0,ORL
1581,2004-05,1610612766,CHA,Charlotte Bobcats,0020400017,2004-11-04,CHA vs. WAS,L,240,96,37,78,0.474,1,8.0,0.125,21,32,0.656,14.0,28.0,42.0,22,6.0,5,19,27,-7.0,WAS
1582,2004-05,1610612766,CHA,Charlotte Bobcats,0010400091,2004-10-26,CHA vs. MIA,W,60,82,33,74,0.446,4,7.0,0.571,12,16,0.750,9.0,32.0,41.0,19,5.0,4,11,25,NaN,MIA
1583,2004-05,1610612766,CHA,Charlotte Bobcats,0010400084,2004-10-24,CHA @ WAS,L,60,85,32,72,0.444,3,12.0,0.250,18,24,0.750,12.0,26.0,38.0,19,12.0,1,24,37,NaN,WAS


In [9]:
counts = mtch['opponent'].value_counts()

In [10]:
mtch[mtch.counts[counts > 21].index

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_14188/1050209549.py, line 1)

In [ ]:
mtch[mtch.]

In [ ]:
mtch[mtch.GAME_ID == '1622100003']